In [1]:
import copy
import numpy as np
import os
import shutil
import pylab
import csv
import sys
import pandas as pd
sys.path.append(os.environ['SU2_RUN'])

import SU2

# libROM snapshot capturing

In [2]:
# setup config file
dir_names = ["data/mach2/",
            "data/mach2p5/",
            "data/mach3/",
            "data/mach3p5/",
            "data/mach4/"]


# input file names:
config_filename = "cyl.cfg"

sol_filename = "restart_flow.csv"

# output file names from librom:
librom_filenames = ["mach2",
                    "mach2p5",
                    "mach3",
                    "mach3p5",
                    "mach4"]

mesh_filename   = "cyl_med_20k.su2" # assumes you are using the same mesh for all
path_to_SU2_CFD = "/Users/jessie/Software/SU2/bin/SU2_CFD" # just "SU2_CFD" if using default SU2 installation

In [4]:
# run SU2 and capture snapshots

for i in range(len(dir_names)):
    config = SU2.io.Config(dir_names[i]+config_filename)
    state  = SU2.io.State()
    config.MESH_FILENAME = mesh_filename
    config.NUMBER_PART = int(2) # for parallel computing
    config.NZONES      = int(1) # for multi-zone simulations
    
    config.RESTART_SOL= "YES"
    config.MATH_PROBLEM= "DIRECT"
    config.READ_BINARY_RESTART= "NO"
    config.SAVE_LIBROM="YES"
    config.ITER= 0
    
    config.SOLUTION_FILENAME    = dir_names[i]+sol_filename
    config.LIBROM_BASE_FILENAME = librom_filenames[i]
    
    try:
        info = SU2.run.direct(config) # errors because of Jupyter but creates config_CFD file anyway
    except:
        run_string = path_to_SU2_CFD + " config_CFD"
        os.system("SU2_CFD config_CFD.cfg")

# Use libROM to read snapshots and compute POD basis
- use combine_samples.C file function to create total POD basis. File comes with libROM installation

In [5]:
# folder location of snapshots above
snap_location = "librom_data/"

In [ ]:
# define path to librom snapshots captured above
librom_snaps = ['data/' + x + '/librom/su2_snapshot' for x in librom_filenames]
librom_snaps

In [4]:
run_string = "./combine_samples "+" ".join(str(x) for x in librom_snaps) + " > librom.out"
run_string

'./combine_samples data/mach2/librom/su2_snapshot data/mach2p15/librom/su2_snapshot data/mach2p25/librom/su2_snapshot data/mach2p35/librom/su2_snapshot data/mach2p5/librom/su2_snapshot data/mach2p6/librom/su2_snapshot data/mach2p75/librom/su2_snapshot data/mach2p85/librom/su2_snapshot data/mach3/librom/su2_snapshot data/mach3p15/librom/su2_snapshot data/mach3p25/librom/su2_snapshot data/mach3p35/librom/su2_snapshot data/mach3p5/librom/su2_snapshot data/mach3p6/librom/su2_snapshot data/mach3p75/librom/su2_snapshot data/mach3p85/librom/su2_snapshot data/mach4/librom/su2_snapshot > librom2.out'

In [5]:
os.system(run_string)


65280

In [13]:
# combine basis samples (NOT a typical option for SU2 users)
run_string2 = "./combine_samples -b "+ "total data/unsteady_m2p5/su2 > librom2.out"
run_string2


'./combine_samples -b total data/unsteady_m2p5/su2 > librom3.out'

In [14]:
os.system(run_string2)

0

### go to matlab and create: 
- POD file, 
- initial snapshot/coord file, and 
- reference snapshot file

# run ROM simulations

In [13]:
# run SU2 and capture snapshots

#~ required inputs ~#
ConfigFileName        = "cyl.cfg"
mesh_filename         = "cyl_med_20k.su2"
POD_filename          = "inputs/POD_mean.csv"
ref_snapshot_filename = "inputs/wref_mean.csv"
iters        = 500
nPOD_modes   = 8
hyper_nodes  = 0      # use 0 for no hyper-reduction

#~ other possible inputs ~#

# if using hyper-reduction:
hyper_nodes_filename = None

# use ONE of the following three:
# if you want to use a restart file as the initial condition:
solution_filename         = None 
# or use one of the following:
initial_snapshot_filename = None
initial_coord_filename    = "inputs/coord_baseline.csv"

# -------------------------------------------------- #

#~ setup new configuration ~#
config = SU2.io.Config(ConfigFileName)
state  = SU2.io.State()
config.MESH_FILENAME = mesh_filename
config.NUMBER_PART = int(1) # for parallel computing
config.NZONES      = int(1) # for multi-zone simulations

#~ setup config file: ~#
if (solution_filename): 
    config.RESTART_SOL = "YES"
else:
    config.RESTART_SOL = "NO"
config.MATH_PROBLEM = "ROM"
config.READ_BINARY_RESTART = "NO"
config.SAVE_LIBROM = "NO"
config.ITER = iters

for i in range(len([1])):
    config.ROM_TESTBASIS_FILENAME = POD_filename
    config.POD_MODES= nPOD_modes
    config.REF_SNAPSHOT_FILENAME = ref_snapshot_filename
    config.CENTRAL_JACOBIAN_FIX_FACTOR = 1.0
    config.TIME_DISCRE_FLOW= "EULER_IMPLICIT"
    
    if (solution_filename): 
        config.SOLUTION_FILENAME = solution_filename
    elif (initial_snapshot_filename): 
        config.INITIAL_SNAPSHOT_FILENAME = initial_snapshot_filename
    elif (initial_coord_filename):
        config.INITIAL_COORD_FILENAME = initial_coord_filename

    config.HYPER_NODES= hyper_nodes
    if (hyper_nodes > 0):
        config.HYPER_NODES_FILENAME = hyper_nodes_filename
    
    try:
        info = SU2.run.direct(config)
    except:
        os.system("/Users/jessie/Software/SU2-nemor/bin/SU2_CFD config_CFD.cfg > a.out")

# save true solution from snapshot

In [20]:
# run SU2 and capture snapshots

ConfigFileName = "data/N2_p7/box.cfg"
solution_filename = "data/N2_p7/restart_flow.csv"
librom_output_name = "wtrue"
mesh_filename = "data/N2_1/box100.su2"

# setup new configuration
config = SU2.io.Config(ConfigFileName)
state  = SU2.io.State()
config.MESH_FILENAME = mesh_filename
config.NUMBER_PART = int(1) # for parallel computing
config.NZONES      = int(1) # for multi-zone simulations

# setup config file:
config.RESTART_SOL= "YES"
config.MATH_PROBLEM= "DIRECT"
config.READ_BINARY_RESTART= "NO"
config.SAVE_LIBROM="YES"
config.ITER= 0

for i in range(len([1])):
    config.SOLUTION_FILENAME    = solution_filename
    config.LIBROM_BASE_FILENAME = librom_output_name
    
    try:
        info = SU2.run.direct(config)
    except:
        os.system("/Users/jessie/Software/SU2-nemor/bin/SU2_CFD config_CFD.cfg > wtrue.out")